In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
import os

import numpy as np
import open3d as o3d
import pandas as pd

In [10]:
from src import Config
from src.common.logger import Logger
from src.feature import LasFileHandler, Voxelizer, VoxelFeatureExtractor
from src.feature.voxelizer.voxel_dto import VoxelDto
from src.feature.point_cloud_file import FileType, Filename, IFileHandler, DirectoryType

In [11]:
logger: Logger = Logger()
point_cloud_file_handler: IFileHandler = LasFileHandler(logger=logger)

## Creating dataset for entire file

In [12]:
def create_dataset(*file_names: Filename, has_label: bool) -> None:
    datasets: list[pd.DataFrame] = []
    if has_label:
        directory_type: DirectoryType = DirectoryType.CROPPED
    else:
        directory_type: DirectoryType = DirectoryType.RAW

    for file_name in file_names:
        point_cloud_file_handler.open(
            directory_type=directory_type,
            filename=file_name,
            file_type=FileType.LAS
        )

        point_cloud: o3d.geometry.PointCloud = point_cloud_file_handler.render_point_cloud_object()
        voxelizer: Voxelizer = Voxelizer(point_cloud=point_cloud, logger=logger)
        voxel_dtos: list[VoxelDto] = voxelizer.voxels
        voxel_feature_extractors: list[tuple[VoxelDto, VoxelFeatureExtractor]] = []

        for dto in voxel_dtos:
            voxel: o3d.geometry.PointCloud = dto.point_cloud
            voxel_feature_extractor: VoxelFeatureExtractor = VoxelFeatureExtractor(point_cloud=voxel, logger=logger)
            voxel_feature_extractors.append((dto, voxel_feature_extractor))

        feature_frame: pd.DataFrame = pd.DataFrame(columns=[
            "grid_x",
            "grid_y",
            "grid_z",

            "r",
            "g",
            "b",

            "max_bound_x",
            "max_bound_y",
            "max_bound_z",

            "min_bound_x",
            "min_bound_y",
            "min_bound_z",

            "l1",
            "l2",
            "l3",
            "planarity",
            "linearity",
            "scattering",
            "omnivariance",
            "sum_of_eigenvalues",
            "eigenentropy",
            "anisotropy",
            "change_of_curvature",
            "z_range",
        ])
        
        for vfe in voxel_feature_extractors:
            voxel_dto: VoxelDto = vfe[0]
            voxel_feature_extractor: VoxelFeatureExtractor = vfe[1]
            
            # Values to store in the dataset
            dataset_values = np.concatenate([
                voxel_dto.grid_index,
                voxel_dto.voxel_color,
                voxel_dto.bounding_box.get_max_bound(),
                voxel_dto.bounding_box.get_min_bound(),
                voxel_feature_extractor.features.array
            ])
            
            insertion_index: int = len(feature_frame)
            feature_frame.loc[insertion_index] = dataset_values

        if has_label:
            feature_frame["label"] = file_name.value

        datasets.append(feature_frame)
        
    dataset: pd.DataFrame = pd.concat(datasets, axis=0)
    dataset = dataset.sample(frac=1)

    filepath: str = os.path.join(Config.DATASETS_DIR.value, "raw_point_cloud_dataset.csv")
    if has_label:
        filepath: str = os.path.join(Config.DATASETS_DIR.value, "labled_dataset.csv")

    dataset.to_csv(path_or_buf=filepath, index=False)

### Training data

In [13]:
filenames: list[Filename] = [Filename.BUILDINGS, Filename.TERRAIN, Filename.RIVERS, Filename.TREES]
# create_dataset(*filenames, has_label=True)

### Test data

In [14]:
create_dataset(Filename.RAW_DATA, has_label=False)

INFO:src.common.logger.logger:Loaded 23299316 points from raw_data.las
INFO:src.common.logger.logger:Rendering point cloud object
INFO:src.common.logger.logger:Rendered point cloud object
INFO:src.common.logger.logger:Voxel grid created with 1302458 voxels.
INFO:src.common.logger.logger:Voxelizer is extracting 1302458 voxels...


Extracting voxels...:   0%|          | 0/1302458 [00:00<?, ?it/s]

KeyboardInterrupt: 